<a href="https://colab.research.google.com/github/Nicolas-Noir/FacturAR/blob/main/Pruebas_FacturAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FacturAR

## Librerias

In [ ]:
# %pip install pdfplumber
!pip install --upgrade transformers

In [ ]:
import pdfplumber

def extract_invoice_text(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() or ""  # Extrae texto si existe
            # Si hay imágenes, procesar con Tesseract
        return text

texto_factura = extract_invoice_text('compra_5.pdf')


In [ ]:
print(texto_factura)

FACTURA
SAUCEDO GUSTAVO ADRIAN
A
ROSARIO SEGURIDAD CASA CENTRAL Nº 0005-00001287
PRESIDENTE PERON 3998, ROSARIO,
SANTA FE Cod. FECHA: 20/03/2025
001
TEL: 3417013094 RESPONSABLE INSCRIPTO(cid:9)(cid:9)CUIT: 20269996901
INICIO ACT.: 31/08/2013(cid:9)(cid:9)ING. BRUTOS: 021-332416-8
SEÑOR/ES: GONZALEZ LUIS ALBERTO IVA: RESPONSABLE INSCRIPTO CUIT: 23168525869
DOMICILIO: CARBIA 966 LOCALIDAD: ROSARIO NORTE
TELEFONOS: 3415 08-6161 PROVINCIA: SANTA FE
CORREO ELECTRONICO: CONDICION PAGO:
OBSERVACIONES:
Código Descripción Cant. Bultos Precio Uni. Sub Total % Sub Total c/
IVA IVA
4048 PANEL HIKVISION HIBRIDO 8+48 ZONAS - Q26133131 1,00 0,00 75.175,50 75.175,50 21,00 90.962,36
El tipo de cambio utilizado es 1 U$D = ARS $ 1.089,5000
SUBTOTAL: $ 75.175,50
IVA 21%: $15.786,86
FECHA VTO: 30/03/2025
TOTAL: $ 90.962,36
CAE: 75121470585035
Generado por www.duxsoftware.com.ar


* subtotal
* total
* iva
* cuit cliente
* cuit vendedor
* razon social
* fecha emision
* n% facuta

In [ ]:
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.0/662.0 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.76.0
    Uninstalling openai-1.76.0:
      Successfully uninstalled openai-1.76.0


In [ ]:
api_key = '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'

In [ ]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=api_key,
)

completion = client.chat.completions.create(
  model="qwen/qwen3-32b:free",
  messages=[
    {
        "role": "system",
      "content": """you are an helpful assistant that works with transcripted text from invoices, the goal is to find and extract the data from the user prompt.

      User prompt example: invoice structure: fecha emision, numero de factura, razon social comprador, Cuit comprador, razon social vendedor, cuit vendedor, subtotal, IVA, impuestos varios, total.
      invoice data: data

      output example: {
        fecha_emision: 23/02/2025,
        numero_factura: 0002-00001543,
        razon_social_comprador: Sancor SRL,
        cuit_comprador:20-44268241-1,
        razon_social_vendedor: jhon anthony kennedy,
        cuit_vendedor: 20-44232760-3,
        subtotal: 62100.20,
        IVA 21: 13041.00,
        IVA 10.5: 0.00,
        IVA 27: 0.00,
        impuestos_varios: 0.00,
        total: 75141.20
      }

      }
      """
    },
    {
      "role": "user",
      "content": """invoice structure: fecha emision, numero de factura, razon social comprador, Cuit comprador, razon social vendedor, cuit vendedor, subtotal, IVA, impuestos varios, total.
      invoice data: """ + texto_factura
    }
  ]
)


In [ ]:
print(completion.choices[0].message.content)

{
  "fecha_emision": "20/03/2025",
  "numero_factura": "0005-00001287",
  "razon_social_comprador": "GONZALEZ LUIS ALBERTO",
  "cuit_comprador": "23-16852586-9",
  "razon_social_vendedor": "SAUCEDO GUSTAVO ADRIAN",
  "cuit_vendedor": "20-26999690-1",
  "subtotal": 75175.50,
  "IVA 21": 15786.86,
  "IVA 10.5": 0.00,
  "IVA 27": 0.00,
  "impuestos_varios": 0.00,
  "total": 90962.36
}
